In [ ]:
import pandas as pd
import datetime

import json as json
from pandas.io.json import json_normalize

import numpy as np
import os
import scipy.optimize as sco
from datetime import datetime

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import pandas_profiling


In [ ]:
array_dataframe = []
dados = pd.DataFrame([])
scoins = []

folder_candle = 'Operacoes'

for folder in os.scandir(folder_candle):
    scoins.append(folder.name)
    for entry in os.scandir(folder):
        if entry.is_file():
            coin_symbol = (os.path.splitext(entry.name)[0])        
            with open(entry.path) as d:            
                coin_hist = json.load(d);
            df = json_normalize(coin_hist);

            df['coin'] = folder.name
            df['amount'] = pd.to_numeric(df['amount'].replace('.',','))
            df['rate'] = pd.to_numeric(df['total'].replace('.',','))
            df['total'] = pd.to_numeric(df['total'].replace('.',','))
            df['date'] = pd.to_datetime(df['date'])            
            df['day'] = pd.to_datetime(df['date'],format='%Y-%m-%d').dt.date
            
            array_dataframe.append(df)       
        
dados = pd.concat(array_dataframe)

dados = dados.replace([np.inf, -np.inf], np.nan)
dados = dados.fillna(0)
#dados.set_index(['date','coin'], inplace= True)
dados.head()

In [ ]:
grouped_data = dados.groupby('coin', as_index=False)

sorted_grouped_data = sorted(
    grouped_data,  # iterates pairs of (key, corresponding subDataFrame)
    key=lambda x: len(x[1]),  # sort by number of rows (len of subDataFrame)
    reverse=True  # largest first
)

grouped_data.describe()

In [ ]:
seg_semestr2017 = grouped_data.apply(lambda g: g[(g['date'] > datetime(2017,6,1)) & (g['date'] < datetime(2018,1,1)) ])
seg_semestr2017.set_index('date', inplace=True)
seg_semestr2017.head()

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

for scoin in scoins:
    dados[dados.coin == scoin].plot(kind='line', x='date', y='amount', figsize=(15,10), ax=ax, label=scoin)    
    
plt.title('Amount x Date')   

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

dados[(dados.coin == 'USDT_BTC') & (dados.type == 'sell')].plot(kind='line', x='date', y='amount', figsize=(15,10), ax=ax, label=scoin)    

for scoin in scoins:
    dados[(dados.coin == scoin) & (dados.type == 'sell')].plot(kind='line', x='date', y='amount', figsize=(15,10), ax=ax, label=scoin+'_SELL')    
    dados[(dados.coin == scoin) & (dados.type == 'buy')].plot(kind='line', x='date', y='amount', figsize=(15,10), ax=ax, label=scoin+'_BUY')    
    
plt.title('Amount x Date')   

In [ ]:
#fig, ax = plt.subplots(figsize=(8,6))

#for scoin in scoins:
#    dados[dados.coin == scoin].plot(kind='bar', x='date', y='amount', figsize=(15,10), ax=ax, label=scoin)    
    
#plt.title('Amount x Date')   